In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, tensor, mesolve, qeye, Qobj, Options

hbar = 1.0545718e-34 
c = 3.0e8  
deff = 3.5842e-29  # C·m
epsilon_0 = 8.854187817e-12  # 真空介电常数，单位：F/m
omega_0 = 384.230406373e12 - 1.2648885163e9 + 100.205e6
delta_87da = 384.2304844685e12 + 4.271676631815181e9 - 229.8518e6
delta_87db = 384.2304844685e12 - 2.563005979089109e9 - 229.8518e6
delta_87ea = 384.2304844685e12 + 4.271676631815181e9 - 72.9112e6
delta_87eb = 384.2304844685e12 - 2.563005979089109e9 - 72.9112e6
delta_87ca = 384.2304844685e12 + 4.271676631815181e9 - 302.0738e6
delta_87fb = 384.2304844685e12 - 2.563005979089109e9 + 193.7407e6
delta_87fa = 384.2304844685e12 + 4.271676631815181e9 + 193.7407e6

omega_a = 0
omega_b = delta_87da - delta_87db  
omega_c = delta_87ca 
omega_d = delta_87da
omega_e = delta_87ea
omega_f = delta_87fa

omega_alpha = np.array([omega_a, omega_b])  
omega_beta = np.array([omega_c, omega_d, omega_e, omega_f]) 

d_87 = np.array([
    [0, np.sqrt(5/24)*deff, np.sqrt(1/8)*deff, 0],
    [0, np.sqrt(1/120)*deff, np.sqrt(1/8)*deff, np.sqrt(1/5)*deff]
])

omega_e_g1=omega_f
omega_e_g2=omega_f-omega_b

beams = [
    {'n':1,'frequency': omega_0+4.08e9,'intensity': 100, 'direction':+1},
    {'n':2,'frequency': omega_0+(4.08+3.036)*1e9, 'intensity': 100*2.01, 'direction': -1},
    {'n':3,'frequency': omega_0+(4.08+6.835)*1e9, 'intensity': 100*3.42, 'direction': -1},
    {'n':4,'frequency': omega_0+(4.08+6.835+3.036)*1e9, 'intensity': 100*4.04, 'direction': +1}
]

def E(I0):
    return np.sqrt(2 * I0 / (c * epsilon_0))

mom_dim = 5
p_list = [-2, -1, 0, 1, 2]
I_mom = qeye(mom_dim)
shift_up = Qobj(np.diag(np.ones(mom_dim-1), 1))
shift_down = Qobj(np.diag(np.ones(mom_dim-1), -1))

atomic_dim = 6  # 2末态 + 4中间态
alpha = [basis(atomic_dim, i) for i in range(2)]  # alpha[0]=a, alpha[1]=b
beta = [basis(atomic_dim, 2+i) for i in range(4)]  # beta[0]=c, beta[1]=d, beta[2]=e, beta[3]=f

H_td = []
detune_threshold = 10e9* (2 * np.pi)  

for beam in beams:
    n=beam['n']
    freq = beam['frequency']
    intensity = beam['intensity']
    direction = beam['direction']
    E_field = E(intensity)
    shift_op = shift_up if direction == +1 else shift_down
    
    for i in range(2): 
        for j in range(4):  
            if d_87[i,j] == 0:
                continue

            transition_freq = omega_beta[j] - omega_alpha[i]
            delta = (freq - transition_freq) * (2 * np.pi)
            
            if abs(delta) > detune_threshold:
                continue
          
            rabi = (d_87[i,j] * E_field) / hbar
            print(n,i,j,rabi*1e-6,delta*1e-6)
            H_absorb = tensor(beta[j] * alpha[i].dag(), shift_op)
            H_td.append([H_absorb, lambda t, args=None, r=rabi, d=delta: (r/2) * np.exp(1j*d*t)])
            
            H_emit = tensor(alpha[i] * beta[j].dag(), shift_op.dag())
            H_td.append([H_emit, lambda t, args=None, r=rabi, d=delta: (r/2) * np.exp(-1j*d*t)])

def calculate_gamma(omega_diff, d):
    return ((2*np.pi*omega_diff)**3 * d**2) / (3 * np.pi * epsilon_0 * hbar * c**3)

gamma_ab = np.array([[calculate_gamma(abs(omega_beta[j] - omega_alpha[i]), d_87[i,j]) 
                     if d_87[i,j] != 0 else 0 
                     for j in range(4)] 
                    for i in range(2)])
c_ops = []
for i in range(2):
    for j in range(4):
        if gamma_ab[i,j] > 0:
            C = np.sqrt(gamma_ab[i,j]) * tensor(alpha[i] * beta[j].dag(), I_mom)
            c_ops.append(C)

psi0 = tensor(alpha[0], basis(mom_dim, 2))  # |a, p=2>
proj_ops = [tensor(alpha[i] * alpha[i].dag(), basis(mom_dim, p) * basis(mom_dim, p).dag())
            for p in range(mom_dim) for i in range(2)]

tlist = np.linspace(0, 1e-6, 100)  # 减少时间范围
options = Options(nsteps=100000, store_states=True, method='bdf')  # 增加 nsteps 并使用 BDF 方法

result = mesolve(H_td, psi0, tlist, c_ops, proj_ops, options=options)

# 可视化结果
plt.figure(figsize=(12, 8))
for i, label in enumerate([f"|g{gs}, p={p}>" for p in p_list for gs in [1, 2]]):
    plt.plot(tlist, result.expect[i], label=label)
plt.xlabel("Time (s)")
plt.ylabel("Population")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Population Dynamics with Momentum Transfer")
plt.tight_layout()
plt.show()

1 0 1 42.567237685079746 -7568.7491995109895
1 0 2 32.97244052967332 -8554.836071530937
1 1 1 8.513447537015946 35374.82816037462
1 1 2 32.97244052967332 34388.74128835467
1 1 3 41.70720483528655 32713.317988143153
2 0 1 60.34947510556761 11507.001393086233
2 0 2 46.74650240724449 10520.914521066286
2 1 1 12.069895021113522 54450.578752971844
2 1 2 46.74650240724449 53464.49188095189
2 1 3 59.13016810137464 51789.06858074038
3 0 1 78.72062281608427 35376.822375061485
3 0 2 60.976732233954 34390.735503041535
4 0 1 85.55908885394827 54452.57296765871
4 0 2 66.27378524960444 53466.48609563876
